In [1]:
import torch
import numpy as np
from utils.dataset import SimpleIterDataset
import os

In [2]:
data_config_name = 'data/AK4/ak4_points_pf_sv_CHS_eta4p7.yaml'

In [3]:
model_state_dict = data_config_name.replace('.yaml','.pt')
jit_model_save = data_config_name.replace('.yaml','_ragged_gpu_jit.pt')
onnx_model = data_config_name.replace('.yaml','.onnx')

In [4]:
data_config = SimpleIterDataset([], data_config_name, for_training=False).config

In [5]:
data_config

In [6]:
import networks.particle_net_pf_sv as network_module

In [7]:
model, model_info = network_module.get_model(data_config, for_inference=True)

In [8]:
model_info

{'input_names': ['pf_points',
  'pf_features',
  'pf_mask',
  'sv_points',
  'sv_features',
  'sv_mask',
  'batch_shapes_pf_points',
  'batch_shapes_pf_features',
  'batch_shapes_pf_mask',
  'batch_shapes_sv_points',
  'batch_shapes_sv_features',
  'batch_shapes_sv_mask'],
 'input_shapes': {'pf_points': (100,),
  'pf_features': (1000,),
  'pf_mask': (50,),
  'sv_points': (10,),
  'sv_features': (55,),
  'sv_mask': (5,),
  'batch_shapes_pf_points': (1, 2),
  'batch_shapes_pf_features': (1, 2),
  'batch_shapes_pf_mask': (1, 2),
  'batch_shapes_sv_points': (1, 2),
  'batch_shapes_sv_features': (1, 2),
  'batch_shapes_sv_mask': (1, 2)},
 'output_names': ['softmax'],
 'dynamic_axes': {'pf_points': {0: 'n_pf'},
  'pf_features': {0: 'n_pf'},
  'pf_mask': {0: 'n_pf'},
  'sv_points': {0: 'n_sv'},
  'sv_features': {0: 'n_sv'},
  'sv_mask': {0: 'n_sv'},
  'batch_shapes_pf_points': {0: 'N'},
  'batch_shapes_pf_features': {0: 'N'},
  'batch_shapes_pf_mask': {0: 'N'},
  'batch_shapes_sv_points': {0:

In [9]:
model.to('cpu')

ParticleNetTagger(
  (pf_conv): FeatureConv(
    (conv): Sequential(
      (0): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv1d(20, 32, kernel_size=(1,), stride=(1,), bias=False)
      (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
  )
  (sv_conv): FeatureConv(
    (conv): Sequential(
      (0): BatchNorm1d(11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv1d(11, 32, kernel_size=(1,), stride=(1,), bias=False)
      (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
  )
  (pn): ParticleNet(
    (edge_convs): ModuleList(
      (0): EdgeConvBlock(
        (convs): ModuleList(
          (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1)

In [10]:
# model.load_state_dict(torch.load(model_state_dict, map_location=torch.device('cpu')))

In [11]:
model.eval()

ParticleNetTagger(
  (pf_conv): FeatureConv(
    (conv): Sequential(
      (0): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv1d(20, 32, kernel_size=(1,), stride=(1,), bias=False)
      (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
  )
  (sv_conv): FeatureConv(
    (conv): Sequential(
      (0): BatchNorm1d(11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv1d(11, 32, kernel_size=(1,), stride=(1,), bias=False)
      (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
  )
  (pn): ParticleNet(
    (edge_convs): ModuleList(
      (0): EdgeConvBlock(
        (convs): ModuleList(
          (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1)

In [12]:
print(model)

ParticleNetTagger(
  (pf_conv): FeatureConv(
    (conv): Sequential(
      (0): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv1d(20, 32, kernel_size=(1,), stride=(1,), bias=False)
      (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
  )
  (sv_conv): FeatureConv(
    (conv): Sequential(
      (0): BatchNorm1d(11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv1d(11, 32, kernel_size=(1,), stride=(1,), bias=False)
      (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
  )
  (pn): ParticleNet(
    (edge_convs): ModuleList(
      (0): EdgeConvBlock(
        (convs): ModuleList(
          (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1)

In [13]:
from train import onnx

In [14]:
model = model.cpu()

In [15]:
model.eval()

ParticleNetTagger(
  (pf_conv): FeatureConv(
    (conv): Sequential(
      (0): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv1d(20, 32, kernel_size=(1,), stride=(1,), bias=False)
      (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
  )
  (sv_conv): FeatureConv(
    (conv): Sequential(
      (0): BatchNorm1d(11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv1d(11, 32, kernel_size=(1,), stride=(1,), bias=False)
      (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
  )
  (pn): ParticleNet(
    (edge_convs): ModuleList(
      (0): EdgeConvBlock(
        (convs): ModuleList(
          (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1)

In [16]:
export_onnx = "ragged_inputs_model.onnx"

In [17]:
    inputs = tuple(
        torch.ones(model_info['input_shapes'][k], dtype=torch.float32) if ('batch_shapes_' not in k) else (torch.tensor([[len(data_config.input_dicts[k.replace('batch_shapes_', '')]), data_config.input_length[k.replace('batch_shapes_', '')]]], dtype=torch.int32)) for k in model_info['input_names'])

In [18]:
inputs

(tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1

In [19]:
print (model_info['input_names'])

['pf_points', 'pf_features', 'pf_mask', 'sv_points', 'sv_features', 'sv_mask', 'batch_shapes_pf_points', 'batch_shapes_pf_features', 'batch_shapes_pf_mask', 'batch_shapes_sv_points', 'batch_shapes_sv_features', 'batch_shapes_sv_mask']


In [20]:
torch.onnx.export(model, inputs, export_onnx,
                  input_names=model_info['input_names'],
                  output_names=model_info['output_names'],
                  dynamic_axes=model_info.get('dynamic_axes', None),
                  opset_version=13)

Shape pf_features -  torch.Size([1, 20, 50])
Shape pf points -  torch.Size([1, 2, 50])
Shape pf mask -  torch.Size([1, 1, 50])
Shape sv features -  torch.Size([1, 11, 5])
Shape sv points -  torch.Size([1, 2, 5])
Shape sv mask shape -  torch.Size([1, 1, 5])


In [21]:
model(*inputs)

Shape pf_features -  torch.Size([1, 20, 50])
Shape pf points -  torch.Size([1, 2, 50])
Shape pf mask -  torch.Size([1, 1, 50])
Shape sv features -  torch.Size([1, 11, 5])
Shape sv points -  torch.Size([1, 2, 5])
Shape sv mask shape -  torch.Size([1, 1, 5])


tensor([[0.1255, 0.1314, 0.1206, 0.1265, 0.1186, 0.1270, 0.1324, 0.1180]],
       grad_fn=<SoftmaxBackward0>)